In [ ]:
library(edgeR)
library(gplots)
library(statmod)

In [ ]:
setwd("/data/anomandaris_data/aswin/irsingh/amit_singh_rna_Seq/Ragini_eukar/Ragini_VeroE6/output/s_species/cTabFiles")


In [ ]:
GenewiseCounts<-read.csv('./only_ref_gene_count_matrix.csv',row.name=1)
GenewiseCounts<-as.matrix(GenewiseCounts)


In [ ]:
sort(as.vector(colnames(GenewiseCounts)))

In [ ]:

GenewiseCounts_backup<-GenewiseCounts
group<- c(rep("Uninfected",3),rep('Infected',3),rep('Infected_treated_GYY4137',3))
df_names<-data.frame(condition=group,name=as.vector(colnames(GenewiseCounts)))
colnames(GenewiseCounts)<-df_names$condition
head(GenewiseCounts)

In [ ]:
columnCombiner<-function(df){
    count<-1
    x<-rle(colnames(df))
    z<-list()
    for (i in 1:length(x$length)){
        if (i==1){
        # print(head(GenewiseCounts[,seq(count,x$length[i])]))
        z<-append(z,list(seq(count,x$length[i])))
        count<-x$length[i]+1
        }else{
            # print(head(GenewiseCounts[,seq(count,count-1+x$length[i])]))
            z<-append(z,list(seq(count,count-1+x$length[i])))
            count<-count+x$length[i]
        }
    }
    return(z)
}

In [ ]:
GG <- rownames(GenewiseCounts)
AA <- columnCombiner(GenewiseCounts)
keep <- c()
for (i in 1:length(AA)) {
  print(i)
  keep <- rowSums(GenewiseCounts[,AA[[i]]]>10)
  GG <- cbind(GG,keep)
}
head(GG)
head(GenewiseCounts)

GG <- as.data.frame(GG)
GG <- GG[-1]
GG1<- apply(GG,2, as.numeric)
rownames(GG1) <- rownames(GG)
head(GG1)
keep <- rowSums(GG1 >= 2)>=1
GenewiseCounts<- GenewiseCounts[keep,]

head(GenewiseCounts)


In [ ]:
y<-DGEList(GenewiseCounts,group=group)


options(digits=3)
y$samples #Optional, to see the samples in the matrix

#Normalization for composition bias- TMM normalization. To normalize for across samples variability. (Wild type and mutant)
y<-calcNormFactors(y)
y$samples


In [ ]:
design <- model.matrix(~0+group)
y<-estimateDisp(y,design,robust=TRUE)
fit<-glmQLFit(y,design,robust=TRUE)
head(fit$coefficients)
logCPM<-cpm(y,log=TRUE)
dirname<-'./'
write.csv(cpm(y),paste(dirname,"CPM_values.csv",sep = "_"))
write.csv(logCPM,paste(dirname,"log_CPM_values.csv",sep = "_"))

In [ ]:
cpmDF<-cpm(y)
pca_result<-pccomp(t(cpmDF),scale=TRUE)
pca_table<-data.frame(pca_result$x)
pca_table$group<-unlist(lapply(colnames(cpmDF),FUN=function(x){strsplit(x,'\\.')[[1]][1]}))
pca_table$group<-rep(c('Uninfected','SARS-CoV-2','SARS-CoV-2+GYY4137'),each=3)
sdev<-pca_result$sdev
explained_variance<-(sdev^2)/sum(sdev^2)
explained_variance<-explained_variance*100

In [ ]:
p<-ggplot(pca_table, aes(x=PC1, y=PC2,color=group,shape=group)) +
        geom_point(size=5)+
        scale_colour_manual(values = c('#e64b35','#4dbbd4','#45a088'))+
        scale_fill_manual(values = c('#e64b35','#4dbbd4','#45a088'))+
        geom_polygon(data = triangle_data, aes(x = x, y = y,fill=group), alpha = 0.2, color = NA,show.legend = F)+
        theme_bw() +
        theme(
        text = element_text(family = 'sans',size=20),
        legend.position = "top", 
        legend.text = element_text(size = 14),
        legend.title = element_text(size = 0),
        plot.title = element_text(size = 8*4, hjust = 0.5),
        panel.background = element_blank(),  # Make panel background transparent
        plot.background = element_blank()    # Make plot background transparent
        )+
        annotate("text", x = 25, y = 60, label = "SARS-CoV-2\n + GYY4137", size = 8, color = "black")+
        annotate("text", x = 25, y = -45, label = "SARS-CoV-2", size = 8, color = "black")+
        annotate("text", x = -105, y = 25, label = "Uninfected", size = 8, color = "black")+
        labs(x=paste('PC1 (',explained_variance[1],')',sep=' '),y=paste('PC2 (',explained_variance[2],')',sep=' '),color='')+
        geom_hline(yintercept = 0,linetype='dashed')+
        geom_vline(xintercept = 0,linetype='dashed')+
        guides(color = guide_legend("Conditions"),
                shape = guide_legend("Conditions"),
                fill=guide_legend("Conditions"))

    ggsave('pcaplot.png',p,dpi = 600,bg = 'transparent')
    print(p)

In [ ]:
uniq_group <- unique(group)
comparisons <- c()
for (i in seq(1,length(uniq_group)-1)) {
  for (j in seq(i+1,length(uniq_group))) {
    if ("Uninfected"==uniq_group[i]){
      x <- paste0("group",uniq_group[j])
      y <- paste0("group",uniq_group[i])
    }else{
      if ("Infected" == uniq_group[i]){
        x <- paste0("group",uniq_group[j])
        y <- paste0("group",uniq_group[i])
      }else{
        x <- paste0("group",uniq_group[i])
        y <- paste0("group",uniq_group[j])
        }
    }
    comparisons <- c(comparisons, paste(x,y,sep = "-"))
  }
}
for (i in comparisons){
  print(i)
}

In [ ]:

dir.create("./comparison_files")
dir.create("./heatmaps_comparisons")

col.pan<-colorpanel(100,"orange red","light cyan","blue")
for (x in comparisons) {
  x
  mvsw<-makeContrasts(x,levels=design)
  # We use QL F-tests. It gives stricter error rate control by accounting for uncertainity in dispersion estimate.
  res<-glmQLFTest(fit,contrast=mvsw)
  W<- topTags(res,n = nrow(res$table))
  write.csv(W$table, file= paste(paste("./comparison_files",dirname,sep = "/"),paste(x,".csv",sep = ""),sep = "_"))
}